# 06 - Train Imported Images

Train a neural network on the images imported in the previous step.

More less unchanged from [Machine Learning is Fun! Part 3: Deep Learning and Convolutional Neural Networks](https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721#.ip7wge3wz). Some bug fixes for Python 3.

In [1]:
from __future__ import division, print_function, absolute_import

# Import tflearn and some helpers
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

import os

In [2]:
import h5py
from sklearn.model_selection import train_test_split

In [3]:
hdf_dataset = "imported_dataset.hdf5"

with h5py.File(hdf_dataset, "r") as fid:
    images = fid["images"].value
    classifications = fid["classifications"].value

In [4]:
# Use sklearn's train_test_split to split out training and testing data.
image_train, image_test, classification_train, classification_test = train_test_split(images, classifications)

In [5]:
# Make sure the data is normalized
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Create extra synthetic training data by flipping, rotating and blurring the
# images on our data set.
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
img_aug.add_random_blur(sigma_max=3.)

In [8]:
# Input is a 32x32 image with 3 color channels (red, green and blue)
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)

# Step 1: Convolution
network = conv_2d(network, 32, 3, activation='relu')

# Step 2: Max pooling
network = max_pool_2d(network, 2)

# Step 3: Convolution again
network = conv_2d(network, 64, 3, activation='relu')

# Step 4: Convolution yet again
network = conv_2d(network, 64, 3, activation='relu')

# Step 5: Max pooling again
network = max_pool_2d(network, 2)

# Step 6: Fully-connected 512 node neural network
network = fully_connected(network, 512, activation='relu')

# Step 7: Dropout - throw away some data randomly during training to prevent over-fitting
network = dropout(network, 0.5)

# Step 8: Fully-connected neural network with two outputs (0=isn't a bird, 1=is a bird) to make the final prediction
network = fully_connected(network, 2, activation='softmax')

# Tell tflearn how we want to train the network
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

In [9]:
run_id = "image-classifier"
# Wrap the network in a model object
model = tflearn.DNN(network,
                    tensorboard_verbose=0,
                    checkpoint_path = os.path.join("/tmp",run_id+".checkpoint.tfl"),
                    max_checkpoints = None,
                    session=None,
                    best_val_accuracy=0.00)

In [ ]:
# Train it! We'll do 100 training passes and monitor it as it goes.
model.fit(image_train,
          classification_train,
          validation_set=(image_test, classification_test),
          n_epoch=100,
          shuffle=True, 
          show_metric=True,
          batch_size=96,
          snapshot_epoch=True,
          run_id=run_id)

model.save(run_id+".tfl")